# Test Model _ver 2

In [ ]:
#!git clone https://github.com/Sh41lD4n1n/FederatedLearning

In [ ]:
!pwd

In [ ]:
#/kaggle/input/federatedlearning/FederatedLearning
#/kaggle/working

In [ ]:
import os
os.chdir("/kaggle/input/federatedlearning/FederatedLearning")

In [ ]:
!pwd

In [ ]:
#!git checkout CNN_model

In [ ]:
import os

os.chdir("./Models/CNN_model")
#!ls
from Model import Model,Optimizer_SGD
from Data import Data

os.chdir("../..")
!ls

from Models.MatrixScheduler.MatrixScheduler_identical import MatrixScheduler_identical
from Models.MatrixScheduler.MatrixScheduler_Uniform import MatrixScheduler_Uniform
from Models.FederatedLearning.Server import server
from Models.StatisticClass import Statistic,array_mean_object

#os.chdir("../Reports")
import matplotlib.pyplot as plt
import numpy as np
#!ls

In [ ]:
import torch.optim as optim

In [ ]:
!pwd

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
!pwd

### Model

In [ ]:
import torchvision
torchvision.datasets.CIFAR10(
        root='/kaggle/input/federatedlearning/FederatedLearning/Models/CNN_model/data/cifar-10-python', train=True, download=False)

In [ ]:
d = Data(split = "noSplit",n_workers=1,environment="kaggle")

In [ ]:
train,test = d.get_data()

plt.figure(figsize=(10, 5))
val,counts = np.unique(train[0].targets,return_counts = True)
val1,counts1 = np.unique(test[0].targets,return_counts = True)

plt.subplot(121)
plt.bar(val,counts)
plt.subplot(122)
plt.bar(val1,counts1)
plt.suptitle("worker 0")
plt.show()

In [ ]:
m = Model("Model")
m.set_optimizer(optim.SGD(m.net.parameters(), lr=0.01))
trainloader = d.get_data_loaders()[0]['trainloader']
testloader = d.get_data_loaders()[0]['testloader']

m.run(epoch = 50,train_loader = trainloader,test_loader = testloader)

In [ ]:
print(m.stat_collector.loss[-1])
print(m.stat_collector.accuracy[-1])
m.stat_collector.loss_plot()
m.stat_collector.acc_plot()

## Federated learning

### Identical split, running with update on each iteration

In [ ]:
def model_creator(w_num):
    name = f"FederatedModel_{w_num}"

    model = Model(name)
    
    #opt = Optimizer_SGD(params=model.net.parameters())
    opt = optim.SGD(model.net.parameters(), lr=0.01)
    preconditioner = None
    scaller = None
    model.set_optimizer(opt)
    #model.set_preconditioner(preconditioner)
    #model.set_scaller(scaller)
    return model

In [ ]:
d = Data(split = "ident",n_workers=1,environment="kaggle")

In [ ]:
trains,tests = d.get_data()

for i,(train,test) in enumerate(zip(trains,tests)):
    plt.figure(figsize=(10, 5))

    val,counts = np.unique(train.indices,return_counts = True)
    val1,counts1 = np.unique(test.indices,return_counts = True)

    plt.subplot(121)
    plt.bar(val,counts)
    plt.subplot(122)
    plt.bar(val1,counts1)
    plt.suptitle(f"worker {i}")
    plt.show()

In [ ]:
s = server(num_workers=1,data = d.get_data_loaders(),model_creator=model_creator)
s.run(n_iter=50,T=1)

In [ ]:
print(s.workers[0].model.stat_collector.loss[-1])
print(s.workers[0].model.stat_collector.accuracy[-1])
s.workers[0].model.stat_collector.loss_plot()
s.workers[0].model.stat_collector.acc_plot()

### Identical split

In [ ]:
def model_creator(w_num):
    name = f"FederatedIdentModel_{w_num}"
    #opt = Optimizer_SGD()
    preconditioner = None
    scaller = None

    model = Model(name)
    #model.set_optimizer(opt)
    #model.set_preconditioner(preconditioner)
    #model.set_scaller(scaller)
    return model

In [ ]:
d = Data(split = "ident",n_workers=3,environment="kaggle")

In [ ]:
trains,tests = d.get_data()

for i,(train,test) in enumerate(zip(trains,tests)):
    plt.figure(figsize=(10, 5))

    val,counts = np.unique(train.indices,return_counts = True)
    val1,counts1 = np.unique(test.indices,return_counts = True)

    plt.subplot(121)
    plt.bar(val,counts)
    plt.subplot(122)
    plt.bar(val1,counts1)
    plt.suptitle(f"worker {i}")
    plt.show()

In [ ]:
s = server(num_workers=3,data = d.get_data_loaders(),model_creator=model_creator)
s.run(n_iter=30,T=5)

In [ ]:
stat_collectors = [w.model.stat_collector for w in s.workers]
stat_collector = array_mean_object(stat_collectors,"ident")

In [ ]:
print(stat_collector.loss[-1])
print(stat_collector.accuracy[-1])
stat_collector.loss_plot()
stat_collector.acc_plot()

## Heterogenious

In [ ]:
def model_creator(w_num):
    name = f"FederatedHetModel_{w_num}"
    #opt = Optimizer_SGD()
    preconditioner = None
    scaller = None

    model = Model(name)
    #model.set_optimizer(opt)
    #model.set_preconditioner(preconditioner)
    #model.set_scaller(scaller)
    return model

In [ ]:
d = Data(split = "het",n_workers=3,environment="kaggle")

In [ ]:
trains,tests = d.get_data()

for i,(train,test) in enumerate(zip(trains,tests)):
    plt.figure(figsize=(10, 5))

    val,counts = np.unique(train.indices,return_counts = True)
    val1,counts1 = np.unique(test.indices,return_counts = True)

    plt.subplot(121)
    plt.bar(val,counts)
    plt.subplot(122)
    plt.bar(val1,counts1)
    plt.suptitle(f"worker {i}")
    plt.show()

In [ ]:
len(trains[0]),len(trains[1]),len(trains[2]),len(trains[0])+len(trains[1])+len(trains[2])

In [ ]:
len(tests[0]),len(tests[1]),len(tests[2]),len(tests[0])+len(tests[1])+len(tests[2])

In [ ]:
s = server(num_workers=3,data = d.get_data_loaders(),model_creator=model_creator)
s.run(n_iter=30,T=5)

## Launch tensorboard

In [ ]:
!pip install tensorboard
#!tensorboard --logdir=runs
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp -r /content/FederatedLearning/runs /content/drive/MyDrive/semester\ 8/Thesis/R1

### Scaller ident

In [ ]:
def model_creator():
    name = "FederatedModel_ident"
    opt = Optimizer_SGD()
    preconditioner = None
    scaller = MatrixScheduler_Uniform(0.1,2,change_mode = MatrixScheduler_Uniform.change_on_each_global_update)

    model = Model(name)
    #model.set_optimizer(opt)
    #model.set_preconditioner(preconditioner)
    model.set_scaller(scaller)
    return model

In [ ]:
d = Data(split = "ident",n_workers=3,environment="kaggle")

In [ ]:
trains,tests = d.get_data()

for i,(train,test) in enumerate(zip(trains,tests)):
    plt.figure(figsize=(10, 5))

    val,counts = np.unique(train.indices,return_counts = True)
    val1,counts1 = np.unique(test.indices,return_counts = True)

    plt.subplot(121)
    plt.bar(val,counts)
    plt.subplot(122)
    plt.bar(val1,counts1)
    plt.suptitle(f"worker {i}")
    plt.show()

In [ ]:
s = server(num_workers=3,data = d.get_data_loaders(),model_creator=model_creator)
s.run(n_iter=200,T=20)